In [14]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score # 정확도 함수
from catboost import CatBoostClassifier
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from xgboost import XGBClassifier


In [48]:
data = pd.read_csv(f'Challenger_Ranked_Games_10minute.csv')
data.columns


Index(['gameId', 'blueWins', 'blueTotalGolds', 'blueCurrentGolds',
       'blueTotalLevel', 'blueAvgLevel', 'blueTotalMinionKills',
       'blueTotalJungleMinionKills', 'blueFirstBlood', 'blueKill', 'blueDeath',
       'blueAssist', 'blueWardPlaced', 'blueWardKills', 'blueFirstTower',
       'blueFirstInhibitor', 'blueFirstTowerLane', 'blueTowerKills',
       'blueMidTowerKills', 'blueTopTowerKills', 'blueBotTowerKills',
       'blueInhibitor', 'blueFirstDragon', 'blueDragnoType', 'blueDragon',
       'blueRiftHeralds', 'redWins', 'redTotalGolds', 'redCurrentGolds',
       'redTotalLevel', 'redAvgLevel', 'redTotalMinionKills',
       'redTotalJungleMinionKills', 'redFirstBlood', 'redKill', 'redDeath',
       'redAssist', 'redWardPlaced', 'redWardKills', 'redFirstTower',
       'redFirstInhibitor', 'redFirstTowerLane', 'redTowerKills',
       'redMidTowerKills', 'redTopTowerKills', 'redBotTowerKills',
       'redInhibitor', 'redFirstDragon', 'redDragnoType', 'redDragon',
       'redRift

In [49]:
winData = data[data['blueWins'] == 1]
loseData = data[data['blueWins'] == 0]
winData = winData.iloc[:, :26]
loseData = loseData.iloc[:, :26]

In [55]:
data = pd.concat([winData, loseData], axis=0, ignore_index=True)
data = data.drop(['gameId'],axis=1)
data = data.drop(['blueDragnoType'],axis=1)
data

,blueWins,blueTotalGolds,blueCurrentGolds,blueTotalLevel,blueAvgLevel,blueTotalMinionKills,blueTotalJungleMinionKills,blueFirstBlood,blueKill,blueDeath,...,blueFirstInhibitor,blueFirstTowerLane,blueTowerKills,blueMidTowerKills,blueTopTowerKills,blueBotTowerKills,blueInhibitor,blueFirstDragon,blueDragon,blueRiftHeralds
0,1,14497,2617,33,6.6,229,44,0,2,3,...,0,0,0,0,0,0,0,0,0,1
1,1,17472,3512,35,7.0,257,46,0,7,5,...,0,0,0,0,0,0,0,0,0,1
2,1,18931,1506,34,6.8,205,48,0,12,11,...,0,0,0,0,0,0,0,0,0,0
3,1,17766,3381,35,7.0,241,36,0,7,7,...,0,0,0,0,0,0,0,0,0,0
4,1,18255,2020,36,7.2,266,56,0,7,9,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26404,0,15484,2094,34,6.8,262,48,0,2,2,...,0,0,0,0,0,0,0,0,0,0
26405,0,17253,1153,32,6.4,196,50,0,9,13,...,0,0,0,0,0,0,0,0,0,0
26406,0,16762,5072,36,7.2,206,52,0,7,5,...,0,0,0,0,0,0,0,0,0,0
26407,0,35765,1440,39,7.8,191,16,0,18,27,...,0,1,2,0,2,0,0,1,1,0


In [56]:
rf = RandomForestClassifier(n_estimators=500, max_depth=50, random_state = 10)
lgbm = LGBMClassifier(n_estimators=100, max_depth=10, verbosity=0, min_data_in_leaf=30, random_state=10)
cat = CatBoostClassifier(iterations=100, depth=4, learning_rate=0.2, random_state=10)
et = ExtraTreesClassifier(n_estimators=100, random_state = 10)
# xgb = XGBClassifier(n_estimators=100, learning_rate=1, max_depth=10, random_state = 10)
xgb = XGBClassifier(n_estimators=100, learning_rate=0.2, max_depth=4, random_state = 32)

In [57]:
# colCnt = data.shape[1]
# print(colCnt)
X = data.iloc[:, 1:]
y = data.iloc[:, :1]


In [58]:

# # features/target, train/test dataset 분리
train_x, test_x, train_y, test_y = train_test_split(X, y, test_size = 0.2, random_state = 42) # 학습데이터와 평가데이터의 비율을 8:2 로 분할|
train_y = train_y.values.ravel()
rf.fit(train_x, train_y)
rf_pre = rf.predict(test_x)
tn, fp, fn, tp = confusion_matrix(test_y, rf_pre).ravel()
print("RandomForest :", round(accuracy_score(test_y, rf_pre)*100, 2), "%")
print("RandomForest f1 :", round(f1_score(test_y, rf_pre)*100, 2), "%")
print('tn:', tn, ' fp:', fp, ' fn:', fn, ' tp:', tp)
lgbm.fit(train_x, train_y)
lgbm_pre = lgbm.predict(test_x)
tn, fp, fn, tp = confusion_matrix(test_y, lgbm_pre).ravel()
print("lightGBM :", round(accuracy_score(test_y, lgbm_pre)*100, 2), "%")
print("lightGBM f1 :", round(f1_score(test_y, lgbm_pre)*100, 2), "%")
print('tn:', tn, ' fp:', fp, ' fn:', fn, ' tp:', tp)
cat.fit(train_x, train_y)
cat_pre = cat.predict(test_x)
tn, fp, fn, tp = confusion_matrix(test_y, cat_pre).ravel()
print("CatBoost :", round(accuracy_score(test_y, cat_pre)*100, 2), "%")
print("CatBoost f1 :", round(f1_score(test_y, cat_pre)*100, 2), "%")
print('tn:', tn, ' fp:', fp, ' fn:', fn, ' tp:', tp)
et.fit(train_x, train_y)
et_pre = et.predict(test_x)
tn, fp, fn, tp = confusion_matrix(test_y, et_pre).ravel()
print("ExtraTrees :", round(accuracy_score(test_y, et_pre)*100, 2), "%")
print("ExtraTrees f1 :", round(f1_score(test_y, et_pre)*100, 2), "%")
print('tn:', tn, ' fp:', fp, ' fn:', fn, ' tp:', tp)
xgb.fit(train_x, train_y)
xgb_pre = xgb.predict(test_x)
tn, fp, fn, tp = confusion_matrix(test_y, xgb_pre).ravel()
print("XGboost :", round(accuracy_score(test_y, xgb_pre)*100, 2), "%")
print("XGboost f1 :", round(f1_score(test_y, xgb_pre)*100, 2), "%")
print('tn:', tn, ' fp:', fp, ' fn:', fn, ' tp:', tp)

RandomForest : 71.9 %
RandomForest f1 : 71.87 %
tn: 1902  fp: 739  fn: 745  tp: 1896
[LightGBM] [Warning] min_data_in_leaf is set=30, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=30
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] min_data_in_leaf is set=30, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=30
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] min_data_in_leaf is set=30, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=30
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
lightGBM : 73.23 %
lightGBM f1 : 73.13 %
tn: 1944  fp: 697  fn: 717  tp: 1924
0:	lea